In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 11.9MB/s 
     |████████████████████████████████| 1.1MB 56.2MB/s 
     |████████████████████████████████| 2.9MB 58.5MB/s 
     |████████████████████████████████| 890kB 40.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0bb7cc57631544b6c0a4d73a45d0219c29ec87a348f42bcf69038d6124f8fc22
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
from tqdm.notebook import tqdm

from transformers import XLNetTokenizer
from torch.utils.data import TensorDataset

from transformers import XLNetForSequenceClassification

In [4]:
import pandas as pd 
df = pd.read_csv('data/edu_data.csv')

In [5]:
df.head()

,content,intent,Intent_number
0,hi,Greetings,0
1,testing the message...plz ingnore,ignore,131
2,hi,Greetings,0
3,Hi For any CNN model to be Oed as LeNet model ...,cnn_question,132
4,Let suppose my CNN model is having same conv l...,cnn_question,132


In [6]:
df.drop(df[df['intent'].map(df['intent'].value_counts()) == 1].index, inplace = True)

In [7]:
df['intent'].value_counts()

need_help                  952
O                          763
Greetings                  408
Thank_you                  353
good_bye                   213
                          ... 
azure_luis_500_error         2
cnn_question                 2
GCP_chatbot                  2
cuda_driver_install          2
books_availablity_drive      2
Name: intent, Length: 114, dtype: int64

In [8]:
df[df['intent'].map(df['intent'].value_counts()) == 1]

,content,intent,Intent_number


In [9]:
possible_labels = df.intent.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Dashboard_access': 22,
 'Dashboard_details': 96,
 'EDA_code': 105,
 'GCP_chatbot': 76,
 'Greetings': 0,
 'NewBatch_details': 42,
 'O': 3,
 'Plotly_installation': 98,
 'Thank_you': 4,
 'anaconda_jnotebook_run': 27,
 'anaconda_macos': 28,
 'anaconda_prompt': 97,
 'assignment_data_visulization': 104,
 'assignment_doubt': 21,
 'assignment_evaluation': 29,
 'assignment_issue': 20,
 'assignment_link': 26,
 'assignment_numpy': 92,
 'assignment_pandas': 83,
 'assignment_pandas_link': 102,
 'assignment_powerbi': 43,
 'assignment_python': 81,
 'assignment_stats': 91,
 'assignment_submittion': 51,
 'azure_luis_500_error': 9,
 'batch_details': 52,
 'books_availablity_drive': 35,
 'bot_timing': 31,
 'canvas_login_problem': 40,
 'career_transition': 56,
 'certificates_enquery': 90,
 'class_link': 58,
 'class_registeration_link': 37,
 'cmd': 64,
 'cnn_question': 2,
 'code_error': 61,
 'colab_help': 79,
 'community_class': 85,
 'community_class_link': 36,
 'course_contents': 100,
 'course_details': 

In [10]:
df['label'] = df.intent.replace(label_dict)

In [11]:
df.head()

,content,intent,Intent_number,label
0,hi,Greetings,0,0
1,testing the message...plz ingnore,ignore,131,1
2,hi,Greetings,0,0
3,Hi For any CNN model to be Oed as LeNet model ...,cnn_question,132,2
4,Let suppose my CNN model is having same conv l...,cnn_question,132,2


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.content.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [13]:
df['data_type'] = ['not_set']*df.shape[0]
df.head()

,content,intent,Intent_number,label,data_type
0,hi,Greetings,0,0,not_set
1,testing the message...plz ingnore,ignore,131,1,not_set
2,hi,Greetings,0,0,not_set
3,Hi For any CNN model to be Oed as LeNet model ...,cnn_question,132,2,not_set
4,Let suppose my CNN model is having same conv l...,cnn_question,132,2,not_set


In [14]:
df.set_index('content', inplace = True)
df.head()

,intent,Intent_number,label,data_type
content,,,,
hi,Greetings,0,0,not_set
testing the message...plz ingnore,ignore,131,1,not_set
hi,Greetings,0,0,not_set
"Hi For any CNN model to be Oed as LeNet model , it should have same number of layers and exact same size/number of filter ? Then only we will O it LeNet model ? Is it ?",cnn_question,132,2,not_set
Let suppose my CNN model is having same conv layer n max pool layer but instead of having 6 filters of 5*5 it is having 6 filter of 3*3 . So is this model can be consider as LeNet or not ?,cnn_question,132,2,not_set


In [15]:

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [16]:

df.groupby(['intent', 'label', 'data_type']).count()

Intent_number
intent                     label data_type               
Dashboard_access           22    train                 19
                                 val                    4
Dashboard_details          96    train                  2
                                 val                    2
EDA_code                   105   train                  2
...                                                   ...
yes_reply                  8     train                 19
                                 val                    4
youtube_video_availability 88    train                  6
                                 val                    1
zoom_mute_disable_reply    10    train                  2

[197 rows x 1 columns]

In [17]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', 
                                          do_lower_case=True)

In [18]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].index.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].index.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [19]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [20]:
len(dataset_train), len(dataset_val)

(3090, 1192)

In [21]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [22]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [23]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [24]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [25]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [26]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [29]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [30]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_xlnet_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 2.726511873348245
Validation loss: 2.078443209692162
F1 Score (Weighted): 0.29683232399070303



Epoch 2
Training loss: 2.209637711096772
Validation loss: 1.6040097933693147
F1 Score (Weighted): 0.5934599895362495



Epoch 3
Training loss: 1.9370146842913034
Validation loss: 1.4877466896745428
F1 Score (Weighted): 0.6150769972260699



In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [ ]:
model.load_state_dict(torch.load('data_volume/finetuned_xlnet_epoch_3.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: VLDB
Accuracy: 1/64

Class: ISCAS
Accuracy: 0/130

Class: SIGGRAPH
Accuracy: 1/49

Class: INFOCOM
Accuracy: 30/77

Class: WWW
Accuracy: 34/57



In [ ]:
# Get some data:

sample = df.sample(n=5)
texts = sample['Title'].to_list()
true_labels = sample['label'].to_list()

In [ ]:
sample

,Title,Conference,label,data_type
62,Analysis of power supply gain of CMOS bandgap ...,ISCAS,1,train
280,Autonomous lighting agents in global illuminat...,SIGGRAPH,2,train
1161,Homepage live: automatic block tracing for web...,WWW,4,val
879,Tunable QoS-aware network survivability.,INFOCOM,3,train
1131,A Protocol-Independent Approach for Analyzing ...,INFOCOM,3,train


In [ ]:
# Model
loaded_model = model.load_state_dict(torch.load('data_volume/finetuned_xlnet_epoch_3.model', map_location=torch.device('cuda')))

In [ ]:
t1 = "Optimized numerical mapping scheme for filter-based exon location in DNA using a quasi-Newton algorithm."

inputs = tokenizer(t1, add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt')



/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
model.eval()

inputs.to(device)
with torch.no_grad():
  outputs = model(**inputs)
# Get the predicted labels
pred_labels = np.argmax(outputs[0].cpu().detach().numpy(), axis=1).tolist()
# print(f"text = {text}")
# print(f"true_labels = {true_labels}")
print(f"pred_labels = {pred_labels}")

for i, j in label_dict.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    if j == pred_labels[0]:
        print(i)

pred_labels = [1]
ISCAS
